In [49]:
import random
import time
import traceback

# Write error handling and retry functionality for 2 functions simulating an an api call to a 3rd party service
# 1. write this for an api call that returns an actual error
# 2. write this for an api call that can return 200 or any other error code


class CodeCommitApiError(Exception):

    def __init__(self):
        print('Error: AWS CodeCommit server is likely down')
        print('try again in a short amount of time')


class S3ApiError(Exception):

    def __init__(self):
        print(f'Error: AWS S3 server is likely down')
        print('try again in a short amount of time')


list_of_custom_errors = ['CodeCommitApiError', 'S3ApiError']


def retry(function_name, retry_count=3):

    def wrapper(*args, **kwargs):
        counter = 0
        while counter <= retry_count:
            try:
                return function_name(*args, **kwargs)
            except Exception as error:
                print(f'Name of error caught: {type(error).__name__}')
                counter += 1
                if counter <= retry_count:
                    print(f"\nRetrying {counter}/{retry_count}:")
                else:
                    print(
                        f"\nMax amount of retries reached: {counter-1}/{retry_count}"
                    )
                    raise
                time.sleep(.1)

    return wrapper


class Deployer():

    def __init__(self):
        self.STEP_COUNT = 0
        self.STEP_MAX = 2

    def next_step_log_message(self):
        self.STEP_MAX = 2
        self.STEP_COUNT += 1
        print(f'\n')
        print(f'Step: {self.STEP_COUNT}/{self.STEP_MAX}')
        print(f'____________________________')

    @retry
    def codecommit_api_call(self,
                            first_number,
                            last_number,
                            error=CodeCommitApiError):
        """case where api returns an actual error,
           using a zero division error here just as a simple example"""

        print(
            f'attempting to divide given numbers: {first_number}/{last_number}'
        )
        print(f'Answer to division: {first_number/last_number}')

    @retry
    def s3_api_call(self, first_number, last_number):
        """ case where api call can return 200 or anything else """
        # api_call_response = api.api_call(arg1, arg 2)
        response_code = random.randint(first_number, last_number)
        if response_code == 200:
            print(f'Success: {response_code}')
            return response_code
        else:
            print(f'Fail: {response_code}')
            raise S3ApiError()


# now this is the "high level function", which is sitting in another file
# ________________________________________________________________
@retry
def high_level_function():
    print('high level function begining')
    deployer = Deployer()
    try:
        deployer.next_step_log_message()
        deployer.codecommit_api_call(5, 1)

        deployer.next_step_log_message()
        deployer.s3_api_call(400, 550)

        print('\n___________________________________________')
        print('high level function completed successfully')
    except Exception:
        print(traceback.format_exc())
        print('base exception caught in higher level function')
        print('\nHigh level function did not complete')
        print('Retrying entire app flow')
        raise
    finally:
        print('Cleaning up temporary files\n\n\n')


print('executing high level function')
high_level_function()

executing high level function
high level function begining


Step: 1/2
____________________________
attempting to divide given numbers: 5/1
Answer to division: 5.0


Step: 2/2
____________________________
Fail: 444
Error: AWS S3 server is likely down
try again in a short amount of time
Name of error caught: S3ApiError

Retrying 1/3:
Fail: 475
Error: AWS S3 server is likely down
try again in a short amount of time
Name of error caught: S3ApiError

Retrying 2/3:
Fail: 496
Error: AWS S3 server is likely down
try again in a short amount of time
Name of error caught: S3ApiError

Retrying 3/3:
Fail: 546
Error: AWS S3 server is likely down
try again in a short amount of time
Name of error caught: S3ApiError

Max amount of retries reached: 3/3
Traceback (most recent call last):
  File "C:\Users\ggomp\AppData\Local\Temp\ipykernel_3436\2232747618.py", line 99, in high_level_function
    deployer.s3_api_call(400, 550)
  File "C:\Users\ggomp\AppData\Local\Temp\ipykernel_3436\2232747618.py", line 3

S3ApiError: 